In [1]:
from bokeh.io import output_notebook
from bokeh.io import show
from bokeh.plotting import figure
import copy
import os
import numpy as np

from pycrysfml import cfml_utilities
#from pycrysfml import crysfml08lib

In [2]:
output_notebook()
FIGURE_WIDTH = 990
FIGURE_HEIGHT = 300

Loading BokehJS ...

In [3]:
os.environ['CRYSFML_DB']

'/Users/andrewsazonov/Development/github.com/EasyScience/TEST_PyCrysFML/.venv/lib/python3.11/site-packages/pycrysfml/Databases'

In [4]:
os.getcwd()

'/Users/andrewsazonov/Development/github.com/EasyScience/TEST_PyCrysFML/examples/pyCFML/cfml_utilities/powder_pattern_from_json'

In [5]:
STUDY_DICT_PM3M = {
  "phases": [
    {
      "SrTiO3": {
        "_space_group_name_H-M_alt": "P m -3 m",
        "_cell_length_a": 3.9,
        "_cell_length_b": 3.9,
        "_cell_length_c": 3.9,
        "_cell_angle_alpha": 90,
        "_cell_angle_beta": 90,
        "_cell_angle_gamma": 90,
        "_atom_site": [
          {
            "_label": "Sr",
            "_type_symbol": "Sr",
            "_fract_x": 0.5,
            "_fract_y": 0.5,
            "_fract_z": 0.5,
            "_occupancy": 1,
            "_adp_type": "Biso",
            "_B_iso_or_equiv": 0.40
          },
          {
            "_label": "Ti",
            "_type_symbol": "Ti",
            "_fract_x": 0,
            "_fract_y": 0,
            "_fract_z": 0,
            "_occupancy": 1,
            "_adp_type": "Biso",
            "_B_iso_or_equiv": 0.50
          },
          {
            "_label": "O",
            "_type_symbol": "O",
            "_fract_x": 0.5,
            "_fract_y": 0,
            "_fract_z": 0,
            "_occupancy": 1,
            "_adp_type": "Biso",
            "_B_iso_or_equiv": 0.65
          }
        ]
      }
    }
  ],
  "experiments": [
    {
      "NPD": {
        "_diffrn_source": "nuclear reactor",
        "_diffrn_radiation_probe": "neutron",
        "_diffrn_radiation_wavelength": 1.27,
        "_pd_instr_resolution_u": 0.02,
        "_pd_instr_resolution_v": -0.02,
        "_pd_instr_resolution_w": 0.12,
        "_pd_instr_resolution_x": 0.0015,
        "_pd_instr_resolution_y": 0,
        "_pd_instr_reflex_asymmetry_p1": 0,
        "_pd_instr_reflex_asymmetry_p2": 0,
        "_pd_instr_reflex_asymmetry_p3": 0,
        "_pd_instr_reflex_asymmetry_p4": 0,
        "_pd_meas_2theta_offset": 0,
        "_pd_meas_2theta_range_min": 1,
        "_pd_meas_2theta_range_max": 140,
        "_pd_meas_2theta_range_inc": 0.05
      }
    }
  ]
}

In [19]:
# Help functions

def generated_x_array(study_dict:dict):
    experiment = study_dict['experiments'][0]['NPD']
    start = experiment['_pd_meas_2theta_range_min']
    stop = experiment['_pd_meas_2theta_range_max']
    step = experiment['_pd_meas_2theta_range_inc']
    x = np.arange(start=start, stop=stop+step, step=step)
    return x

def compute_pattern(study_dict:dict):
    _, y = cfml_utilities.powder_pattern_from_json(study_dict)  # returns x and y arrays
    #_, y = crysfml08lib.f_powder_pattern_from_json(study_dict)  # returns x and y arrays
    return y

In [20]:
def compute_pattern__SrTiO3_Pm3m():
    study_dict = copy.deepcopy(STUDY_DICT_PM3M)

    # x array
    x = generated_x_array(study_dict)

    # desired y array
    _, desired = np.loadtxt('desired/srtio3-pm3m-pattern_Nebil-ifort.xy', unpack=True)
    desired = desired - 20.0  # remove background
    desired = np.roll(desired, -1)  # compensate for a 1-element shift in y data between old Nebil windows build and Andrew current gfortran build
    
    # set space group
    study_dict['phases'][0]['SrTiO3']['_space_group_name_H-M_alt'] = 'P m -3 m'

    # actual y array
    actual = compute_pattern(study_dict)
    
    # compare results
    fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
    fig.line(x, desired, legend_label='desired', color='orangered', line_width=5)
    fig.line(x, actual, legend_label='actual', color='steelblue', line_width=2)
    fig.line(x, actual-desired, legend_label='diff', color='green', line_width=1)
    show(fig)

In [21]:
def compute_pattern__SrTiO3_Pnma():
    study_dict = copy.deepcopy(STUDY_DICT_PM3M)

    # x array
    x = generated_x_array(study_dict)

    # desired y array
    desired = np.loadtxt('desired/srtio3-pnma-pattern_Andrew-ifort.y', unpack=True)
    desired = desired - 20.0  # remove background
    
    # set space group
    study_dict['phases'][0]['SrTiO3']['_space_group_name_H-M_alt'] = 'P n m a'

    # actual y array
    actual = compute_pattern(study_dict)
    
    # compare results
    fig = figure(width=FIGURE_WIDTH, height=FIGURE_HEIGHT)
    fig.line(x, desired, legend_label='desired', color='orangered', line_width=5)
    fig.line(x, actual, legend_label='actual', color='steelblue', line_width=2)
    fig.line(x, actual-desired, legend_label='diff', color='green', line_width=1)
    show(fig)

In [22]:
compute_pattern__SrTiO3_Pm3m()

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [23]:
compute_pattern__SrTiO3_Pnma()

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
